In [1]:
import pandas as pd

In [2]:
import re

### Задание 1

In [3]:
def classification(row):
    if row['rating'] <= 2:
        return 'низкий рейтинг'
    elif row['rating'] <= 4:
        return 'средний рейтинг'
    else:
        return 'высокий рейтинг'

movies = pd.read_csv('ratings.csv')
movies['class'] = movies.apply(classification, axis = 1)
movies.head(10)

,userId,movieId,rating,timestamp,class
0,1,31,2.5,1260759144,средний рейтинг
1,1,1029,3.0,1260759179,средний рейтинг
2,1,1061,3.0,1260759182,средний рейтинг
3,1,1129,2.0,1260759185,низкий рейтинг
4,1,1172,4.0,1260759205,средний рейтинг
5,1,1263,2.0,1260759151,низкий рейтинг
6,1,1287,2.0,1260759187,низкий рейтинг
7,1,1293,2.0,1260759148,низкий рейтинг
8,1,1339,3.5,1260759125,средний рейтинг
9,1,1343,2.0,1260759131,низкий рейтинг


### Задание 2

Для учета различных форм написания городов, применены:
1) регулярные выражения

In [4]:
geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

def geo_classificator(row):
    for key, value in geo_data.items():
        for city in value:
            if key == 'Центр':
                pattern = r'(\s|\b)(' + city[0:-1] + ')([а-яА-Я]){1,}'
                if re.findall(pattern, row['keyword']) != []:
                    if 'тулуп' not in row['keyword'] and 'тулун' not in row['keyword'] and 'пряник' not in row['keyword']:
                        return key
            else:
                pattern = r'(\s|\b)(' + city + ')([а-яА-Я]){0,}'
                if re.findall(pattern, row['keyword']) != []:
                    return key

keywords = pd.read_csv('keywords.csv')
keywords['region'] = keywords.apply(geo_classificator, axis = 1).fillna('undefined')
# проверка
check_keywords_dataset = keywords[keywords['keyword'].str.contains('петербург')]
check_keywords_dataset.head(10)

,keyword,shows,region
370,авито ру санкт петербург,425134,Северо-Запад
564,погода в санкт петербурге,292489,Северо-Запад
1236,банк санкт петербург,174375,Северо-Запад
1346,санкт петербург,144232,Северо-Запад
2121,электронный дневник санкт петербургского образ...,94835,Северо-Запад
2550,банк санкт петербург интернет банк,89123,Северо-Запад
2974,авто ру санкт петербург,74918,Северо-Запад
4492,авито санкт петербурге,53715,Северо-Запад
4654,леруа мерлен каталог товаров в санкт петербурге,52087,Северо-Запад
5045,емлс недвижимость санкт петербурга,48409,Северо-Запад


2) библиотека pymystem (дождаться окончания работы pymystem не удалось :( )

In [ ]:
! pip install pymystem3

In [ ]:
from pymystem3 import Mystem
m = Mystem()

In [ ]:
geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

def lemmatization(row):
    return m.lemmatize(row['keyword'])

def geo_classificator(row):
    for key, value in geo_data.items():
        for city in value:
            if city in row['lemmas']:
                return key

keywords = pd.read_csv('keywords.csv')
keywords['lemmas'] = keywords.apply(lemmatization, axis = 1)
keywords['region'] = keywords.apply(geo_classificator, axis = 1).fillna('undefined')
keywords.head(10)

### Задание 3

In [5]:
years = list(range(1950,2011))

def production_year(row):
    for year in years:
        if str(year) in row['title']:
            return str(year)
        
movies = pd.read_csv('movies.csv')
movies['year'] = movies.apply(production_year, axis = 1).fillna('1900')
filtered_movies = movies[(movies['year'] != '1900')]

ratings = pd.read_csv('ratings.csv')

ranked_movies = ratings.merge(filtered_movies, on = 'movieId', how = 'inner')
ranked_movies.groupby('year').agg({'rating': 'mean'}).sort_values(by = 'rating', ascending = False).head(20)

,rating
year,
1957,4.014241
1972,4.011136
1952,4.000000
1954,3.994220
1951,3.983539
1974,3.978704
1962,3.952446
1950,3.915254
1977,3.905786
